### Imports etc

In [1]:
import sys
from pathlib import Path
import pandas as pd
import seaborn as sns

REPO_PATH = Path.home() / "workspace/jetpointnet"
SCRIPT_PATH = REPO_PATH / "python_scripts"
sys.path.append(str(SCRIPT_PATH))

import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# from mpl_toolkits.axes_grid1 import make_axes_locatable
from jets_training.models.JetPointNet import PointNetSegmentation
from jets_training.jets_visualize import generate_images_and_metadata
from jets_training.jets_train import (
    TRAIN_INPUTS,
    MAX_SAMPLE_LENGTH,
    baseline_configuration,
    EXPERIMENT_NAME,
    TRAIN_TARGETS
)
OUTPUT_ACTIVATION_FUNCTION = baseline_configuration['OUTPUT_ACTIVATION_FUNCTION']




2024-07-30 19:29:44.499017: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 19:29:44.592840: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 19:29:44.592921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 19:29:44.595115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 19:29:44.609330: I tensorflow/core/platform/cpu_feature_guar

Logged in as jhimmens


# Generate Event Images

In [2]:
model_path = "/home/jhimmens/workspace/jetpointnet/models/rev_6/collected_data/PointNet_epoch=10_name=distinctive-star-975.keras"
sets_to_visualize = ["rho", 'delta', "JZ4"]
max_events_per_set = 10_000
max_images_per_set = 40

MODEL_NAME = model_path.split('/')[-1]
VISUAL_PATH = REPO_PATH / "visualizations" / EXPERIMENT_NAME / MODEL_NAME

model = PointNetSegmentation(MAX_SAMPLE_LENGTH, 
                             num_features=len(TRAIN_INPUTS), 
                             num_classes=len(TRAIN_TARGETS), 
                             output_activation_function=OUTPUT_ACTIVATION_FUNCTION,
                             model_version=1)

model.load_weights(model_path)

generate_images_and_metadata(sets_to_visualize, model, max_events_per_set, max_images_per_set, VISUAL_PATH)

2024-07-30 19:29:51.849669: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-30 19:29:51.849762: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: triumf-ml2
2024-07-30 19:29:51.849773: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: triumf-ml2
2024-07-30 19:29:51.850031: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: INVALID_ARGUMENT: expected %d.%d, %d.%d.%d, or %d.%d.%d.%d form for driver version; got "1"
2024-07-30 19:29:51.850093: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.90.7


  0%|          | 0/3 [00:00<?, ?it/s]

rho
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_7/SavedNpz/deltaR=0.2_maxLen=800_MaxTrackAtributions=26_EScale=1/train


rho: 0it [00:00, ?it/s]

/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_7/SavedNpz/deltaR=0.2_maxLen=800_MaxTrackAtributions=26_EScale=1/train/rho/user.mswiatlo.40097513._000010.mltree.root_chunk_23_train.parquet.npz
10/10 [==============================] - 5s 235ms/step
saving: event=000451924_track=000
saving: event=000451933_track=000
saving: event=000451905_track=000
saving: event=000451953_track=000
saving: event=000451898_track=000
saving: event=000451961_track=000
saving: event=000451957_track=000
saving: event=000451959_track=000
saving: event=000451967_track=000
saving: event=000451956_track=000
saving: event=000451972_track=000
saving: event=000451975_track=000
saving: event=000451938_track=000
saving: event=000451951_track=000
saving: event=000451971_track=000
saving: event=000451989_track=000
saving: event=000451991_track=000
saving: event=000451982_track=000
saving: event=000451986_track=000
saving: event=000451996_track=000
saving: event=000451976_

delta: 0it [00:00, ?it/s]

/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_7/SavedNpz/deltaR=0.2_maxLen=800_MaxTrackAtributions=26_EScale=1/train/delta/user.mswiatlo.40097496._000012.mltree.root_chunk_32_train.parquet.npz
11/11 [==============================] - 2s 213ms/step
saving: event=000312187_track=000
saving: event=000312191_track=000
saving: event=000312183_track=000
saving: event=000312193_track=000
saving: event=000312207_track=000
saving: event=000312196_track=000
saving: event=000312196_track=001
saving: event=000312213_track=000
saving: event=000312215_track=000
saving: event=000312201_track=000
saving: event=000312219_track=000
saving: event=000312227_track=000
saving: event=000312247_track=000
saving: event=000312225_track=000
saving: event=000312249_track=000
saving: event=000312259_track=000
saving: event=000312240_track=000
saving: event=000312240_track=001
saving: event=000312276_track=000
saving: event=000312275_track=000
saving: event=00031227

JZ4: 0it [00:00, ?it/s]

/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_7/SavedNpz/deltaR=0.2_maxLen=800_MaxTrackAtributions=26_EScale=1/train/JZ4/user.mswiatlo.39955735._000521.mltree.root_chunk_0_train.parquet.npz
269/269 [==============================] - 47s 174ms/step
saving: event=000035729_track=000
saving: event=000035729_track=001
saving: event=000035729_track=002
saving: event=000035729_track=003
saving: event=000035729_track=005
saving: event=000035729_track=007
saving: event=000035729_track=009
saving: event=000035729_track=010
saving: event=000035729_track=011
saving: event=000035729_track=012
saving: event=000035729_track=016
saving: event=000035729_track=018
saving: event=000035729_track=020
saving: event=000035729_track=022
saving: event=000035729_track=023
saving: event=000035729_track=024
saving: event=000035729_track=025
saving: event=000035729_track=028
saving: event=000035729_track=030
saving: event=000035729_track=038
saving: event=00003572